In [ ]:
import torch
weights = torch.load('save_data/save.pt', map_location=torch.device('cpu'))
import os
if not os.path.exists('./pre_weights/') : os.mkdir('./pre_weights/')

In [ ]:
import joblib
import numpy as np
masks = np.array(joblib.load('./save_data/input_mask0.pkl').cpu(),dtype=np.complex64)
input = np.array(joblib.load('./save_data/input0.pkl').cpu(),dtype=np.complex64)
output = np.array(joblib.load('./save_data/output0.pkl').cpu(),dtype=np.complex64)
label = np.array(joblib.load('./save_data/label0.pkl').cpu())
logit = np.array(joblib.load('./save_data/logit0.pkl').cpu(),dtype=np.complex64)
wout = np.array(weights['classifier.weight'].T,dtype=np.complex64)
bout = np.array(weights['classifier.bias'],dtype=np.complex64)
print(masks.shape,input.shape,output.shape,wout.shape,bout.shape)

In [ ]:
from base_fncs import * 

In [ ]:
import numpy as np
shape = (256,128)

def load_full(load=False, verbose=True) :
    from layers.bert_layer import bert_full
    bert = bert_full(shape,verbose)
    if load : 
        bert.make_weights()
        bert.set_bts_range()
    else : 
        bert.make_weights(weights)
        bert.set_bts_range(input, masks)
    bert.premake()
    return bert

def full_chk(bert) :
    x = input[0,]
    mask = masks[0,]
    packed_x = [column_wise_pack(x[:,i*256:(i+1)*256], 15) for i in range(3)]   
    bert.calc_diff(x,packed_x,mask)

def full_test(bert) :
    calc = []
    tot_times = []
    bts_times = []
    for i in range(len(input)) :
        x = input[i,]
        mask = masks[i,]
        packed_x = [column_wise_pack(x[:,i*256:(i+1)*256], 15) for i in range(3)]   
        now,times,bts_time = bert.forward(packed_x,mask)
        tot_times.append(times)
        bts_times.append(bts_time)
        res = np.concatenate([unpack3(data,shape) for data in now],axis=1)[0,:]
        calc.append(res)
        if i % 50 == 0 : 
            print(i)
            print(np.mean(tot_times,axis=0),np.mean(bts_times,axis=0))
            os.makedirs('./eval_res/')
            joblib.dump(calc,f'./eval_res/acc{i}.pkl')
            joblib.dump(bts_times,f'./eval_res/layer_time{i}.pkl')
            joblib.dump(bts_times,f'./eval_res/bts_time{i}.pkl')
    calc = np.array(calc)
    tot_times = np.array(tot_times)
    bts_times = np.array(bts_times)
    joblib.dump(calc,f'./eval_res/acc_total.pkl')
    joblib.dump(tot_times,f'./eval_res/layer_time_total.pkl')
    joblib.dump(tot_times,f'./eval_res/bts_time_total.pkl')
    logit_now = calc @ wout + bout
    print('he,plain diff',np.max(np.abs(logit-logit_now)))
    acc = np.mean(np.where(label==np.argmax(logit,axis=1),1,0))
    print('real acc',acc)  
    acc = np.mean(np.where(label==np.argmax(logit_now,axis=1),1,0))
    print('he acc',acc)  
    print('mean time', np.mean(tot_times,axis=0))

In [ ]:
bert = load_full(load=False,verbose=True) #load model and precompute

In [ ]:
full_chk(bert)  ## check accuracy for first data

In [ ]:
full_test(bert) ## test for full rte data